# Intakes and Outcomes Merge

#### Imports

In [46]:
import pandas as pd

#### Read in Data

In [47]:
intakes = pd.read_csv('../data/intakes_clean.csv', parse_dates=['datetime'])

In [48]:
outcomes = pd.read_csv('../data/outcomes_cleaned.csv', parse_dates=['datetime'])

#### Merge Datasets

In [49]:
outcomes_intakes = outcomes.merge(intakes, 
                                  left_on=['animal_id', 'outcome_count'],
                                  right_on=['animal_id', 'intake_count'],
                                  suffixes=('_outcome', '_intake'))

#### Filter Dataset

In [53]:
dogs = outcomes_intakes[outcomes_intakes['animal_type_outcome'] == 'Dog']

#### Save csv

In [54]:
dogs.to_csv('../data/dogs.csv', index=False)